# Importing Required Libraries

In [ ]:
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import *
import tensorflow as tf
from keras.callbacks import EarlyStopping
import os
import numpy as np
from keras.models import Sequential
from sklearn.metrics import classification_report

# Helper Functions

In [ ]:
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 16
DATASET_DIR = '' # Change this to the path of the dataset
CLASSES_LIST = ['Brawl', 'Peace']

In [ ]:
def frames_extraction(video_path):
    frames_list = []
    video_reader = cv2.VideoCapture(video_path)
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read()
        if not success:
            break
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame / 255
        frames_list.append(normalized_frame)

    video_reader.release()
    return frames_list

In [ ]:
def create_dataset(DATASET_DIR):
    features = []
    labels = []
    video_files_paths = []

    for class_index, class_name in enumerate(CLASSES_LIST):
        print(f'Extracting Data of Class: {class_name}')
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        for file_name in files_list:
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
            frames = frames_extraction(video_file_path)
            if len(frames) == SEQUENCE_LENGTH:
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    features = np.asarray(features)
    labels = np.array(labels)
    
    return features, labels, video_files_paths

# Extracting Features

In [ ]:
features, labels, video_files_paths = create_dataset(DATASET_DIR=DATASET_DIR)
one_hot_encoded_labels = to_categorical(labels)
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.2, shuffle = True, random_state = 42)

# Model

In [ ]:
mobilenet = MobileNetV2( include_top=False , weights="imagenet")
mobilenet.trainable=True
for layer in mobilenet.layers[:-40]:
    layer.trainable=False

In [ ]:
def create_model():
    model = Sequential()
    model.add(Input(shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    model.add(TimeDistributed(mobilenet))
    model.add(TimeDistributed(Flatten()))
    lstm_fw = LSTM(units=64)
    lstm_bw = LSTM(units=64, go_backwards = True)
    model.add(Bidirectional(lstm_fw, backward_layer = lstm_bw))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(8,activation='relu'))
    model.add(Dense(4,activation='relu'))
    model.add(Dense(len(CLASSES_LIST), activation = 'softmax'))

In [ ]:
model = create_model()
early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 10, restore_best_weights = True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=5, min_lr=0.0000005, verbose=1)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
MobBiLSTM_model_history = model.fit(x = features_train, y = labels_train, epochs = 50, batch_size = 4, shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback,reduce_lr])

# Final Predicting

In [ ]:
PREDICT_DATASET_DIR = '' # Change this to the path of the predict dataset
predict_features, predict_labels, predict_video_files_paths = create_dataset(DATASET_DIR=PREDICT_DATASET_DIR)
predict_labels = to_categorical(predict_labels)

pred = model.predict(predict_features)
predicted_classes = np.argmax(pred, axis = 1)
print(classification_report(np.argmax(predict_labels, axis = 1), predicted_classes, target_names = CLASSES_LIST))